In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import networkx as nx
import random
import time
import pickle as cp
import sys
from tqdm import tqdm
import PrepareBatchGraph
import graph
import nstep_replay_mem
import nstep_replay_mem_prioritized
import mvc_env
import utils
import heapq
import scipy.linalg as linalg
import os
import pandas as pd

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
"""
rewrited tf to pytorch methods

old code is commented out as ref, for example:

#y_input_message = tf.matmul(tf.cast(y_node_input,tf.float32), w_n2l)
y_input_message = torch.matmul(self.y_node_input.type(torch.FloatTensor), w_n2l)

Prepared params such as:
y_nodes_size = tf.shape(self.subgsum_param)[0]

are kept as-is, since I want to test if this model will run at the end of rewrite functions and layers.

"""

In [12]:
class Model(nn.Module):
    def __init__(self, embedding_size, w_initialization_std):
        super(Model, self).__init__()
        
        self.rand_generator = torch.normal
        self.embedding_size = embedding_size
        self.w_initialization_std = w_initialization_std
        
        # [2, embed_dim]
        self.w_n2l = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                     size=(2, self.embedding_size)))
        # [embed_dim, embed_dim]
        self.p_node_conv = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, self.embedding_size)))
        
        if embeddingMethod == 1:    #'graphsage'
            # [embed_dim, embed_dim]
            self.p_node_conv2 = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, self.embedding_size)))
            # [2*embed_dim, embed_dim]
            self.p_node_conv3 = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(2*self.embedding_size, self.embedding_size)))

        #[reg_hidden+aux_dim, 1]
        if self.reg_hidden > 0:
            # [embed_dim, reg_hidden]
            #h1_weight = tf.Variable(tf.truncated_normal([self.embedding_size, self.reg_hidden], stddev=initialization_stddev), tf.float32)
            h1_weight = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, self.reg_hidden)))
            
            #[reg_hidden+aux_dim, 1]
            #h2_weight = tf.Variable(tf.truncated_normal([self.reg_hidden + aux_dim, 1], stddev=initialization_stddev), tf.float32)
            h2_weight = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.reg_hidden + aux_dim, 1)))
            #[reg_hidden2 + aux_dim, 1]
            last_w = h2_weight
        else:
            #[2*embed_dim, reg_hidden]
            #h1_weight = tf.Variable(tf.truncated_normal([2 * self.embedding_size, self.reg_hidden], stddev=initialization_stddev), tf.float32)
            h1_weight = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(2*self.embedding_size, self.reg_hidden)))            
            #[2*embed_dim, reg_hidden]
            last_w = h1_weight
            
        ## [embed_dim, 1]
        #cross_product = tf.Variable(tf.truncated_normal([self.embedding_size, 1], stddev=initialization_stddev), tf.float32)
        cross_product = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, 1)))


    def train_forward(self, node_input):

        #x = torch.matmul(node_input, self.w_n2l)
        #x = F.relu(x)
        
        y_nodes_size = tf.shape(self.subgsum_param)[0]
        # [batch_size, 2]
        #y_node_input = tf.ones((y_nodes_size,2))
        y_node_input = np.ones((y_nodes_size,2))

        #[node_cnt, 2] * [2, embed_dim] = [node_cnt, embed_dim]
        # no sparse
        #input_message = tf.matmul(tf.cast(self.node_input,tf.float32), w_n2l)
        input_message = torch.matmul(self.node_input.type(torch.FloatTensor), w_n2l)
        #[node_cnt, embed_dim]  # no sparse
        #input_potential_layer = tf.nn.relu(input_message)
        input_potential_layer = F.relu(input_message)

        # # no sparse
        # [batch_size, embed_dim]
        #y_input_message = tf.matmul(tf.cast(y_node_input,tf.float32), w_n2l)
        y_input_message = torch.matmul(self.y_node_input.type(torch.FloatTensor), w_n2l)
        #[batch_size, embed_dim]  # no sparse
        #y_input_potential_layer = tf.nn.relu(y_input_message)
        y_input_potential_layer = F.relu(y_input_message)

        #input_potential_layer = input_message
        lv = 0
        #[node_cnt, embed_dim], no sparse
        cur_message_layer = input_potential_layer
        #cur_message_layer = tf.nn.l2_normalize(cur_message_layer, axis=1)
        cur_message_layer = torch.nn.functional.normalize(cur_message_layer, p=2, dim=1)

        #[batch_size, embed_dim], no sparse
        y_cur_message_layer = y_input_potential_layer
        # [batch_size, embed_dim]
        #y_cur_message_layer = tf.nn.l2_normalize(y_cur_message_layer, axis=1)
        y_cur_message_layer = torch.nn.functional.normalize(y_cur_message_layer, p=2, dim=1)

        # max_bp_iter=3  
        while lv < max_bp_iter:
            lv = lv + 1
            #[node_cnt, node_cnt] * [node_cnt, embed_dim] = [node_cnt, embed_dim], dense
            #n2npool = tf.sparse_tensor_dense_matmul(tf.cast(self.n2nsum_param,tf.float32), cur_message_layer)
            # see https://discuss.pytorch.org/t/sparse-tensors-in-pytorch/859/4
            n2npool = torch.matmul(self.n2nsum_param.type(torch.FloatTensor), cur_message_layer)
            #[node_cnt, embed_dim] * [embed_dim, embed_dim] = [node_cnt, embed_dim], dense
            #node_linear = tf.matmul(n2npool, p_node_conv)
            node_linear = torch.matmul(n2npool, p_node_conv)

            # [batch_size, node_cnt] * [node_cnt, embed_dim] = [batch_size, embed_dim]
            #y_n2npool = tf.sparse_tensor_dense_matmul(tf.cast(self.subgsum_param,tf.float32), cur_message_layer)
            # why cur_message_layer, instead of y_cur_message_layer?
            y_n2npool = torch.matmul(self.subgsum_param.type(torch.FloatTensor), cur_message_layer)
            #[batch_size, embed_dim] * [embed_dim, embed_dim] = [batch_size, embed_dim], dense
            #y_node_linear = tf.matmul(y_n2npool, p_node_conv)
            y_node_linear = torch.matmul(y_n2npool, p_node_conv)

            if embeddingMethod == 0: # 'structure2vec'
                #[node_cnt, embed_dim] + [node_cnt, embed_dim] = [node_cnt, embed_dim], return tensed matrix
                #merged_linear = tf.add(node_linear,input_message)
                merged_linear = torch.add(node_linear,input_message)
                #[node_cnt, embed_dim]
                #cur_message_layer = tf.nn.relu(merged_linear)
                cur_message_layer = F.relu(merged_linear)

                #[batch_size, embed_dim] + [batch_size, embed_dim] = [batch_size, embed_dim], return tensed matrix
                #y_merged_linear = tf.add(y_node_linear, y_input_message)
                y_merged_linear = torch.add(y_node_linear, y_input_message)
                #[batch_size, embed_dim]
                #y_cur_message_layer = tf.nn.relu(y_merged_linear)
                y_cur_message_layer = F.relu(y_merged_linear)
                
            else:   # 'graphsage'
                #[node_cnt, embed_dim] * [embed_dim, embed_dim] = [node_cnt, embed_dim], dense
                #cur_message_layer_linear = tf.matmul(tf.cast(cur_message_layer, tf.float32), p_node_conv2)
                cur_message_layer_linear = torch.matmul(cur_message_layer.type(torch.FloatTensor), p_node_conv2)
                #[[node_cnt, embed_dim] [node_cnt, embed_dim]] = [node_cnt, 2*embed_dim], return tensed matrix
                #merged_linear = tf.concat([node_linear, cur_message_layer_linear], 1)
                merged_linear = torch.concat([node_linear, cur_message_layer_linear], 1)
                #[node_cnt, 2*embed_dim]*[2*embed_dim, embed_dim] = [node_cnt, embed_dim]
                #cur_message_layer = tf.nn.relu(tf.matmul(merged_linear, p_node_conv3))
                cur_message_layer = F.relu(torch.matmul(merged_linear, p_node_conv3))

                #[batch_size, embed_dim] * [embed_dim, embed_dim] = [batch_size, embed_dim], dense
                #y_cur_message_layer_linear = tf.matmul(tf.cast(y_cur_message_layer, tf.float32), p_node_conv2)
                y_cur_message_layer_linear = torch.matmul(y_cur_message_layer.type(torch.FloatTensor), p_node_conv2)
                #[[batch_size, embed_dim] [batch_size, embed_dim]] = [batch_size, 2*embed_dim], return tensed matrix
                #y_merged_linear = tf.concat([y_node_linear, y_cur_message_layer_linear], 1)
                y_merged_linear = torch.concat([y_node_linear, y_cur_message_layer_linear], 1)
                #[batch_size, 2*embed_dim]*[2*embed_dim, embed_dim] = [batch_size, embed_dim]
                #y_cur_message_layer = tf.nn.relu(tf.matmul(y_merged_linear, p_node_conv3))
                y_cur_message_layer = F.relu(torch.matmul(y_merged_linear, p_node_conv3))

            #cur_message_layer = tf.nn.l2_normalize(cur_message_layer, axis=1)
            #y_cur_message_layer = tf.nn.l2_normalize(y_cur_message_layer, axis=1)
            cur_message_layer = torch.nn.functional.normalize(cur_message_layer, p=2, dim=1)
            y_cur_message_layer = torch.nn.functional.normalize(y_cur_message_layer, p=2, dim=1)

        
        self.node_embedding = cur_message_layer
        #[batch_size, node_cnt] * [node_cnt, embed_dim] = [batch_size, embed_dim], dense
        y_potential = y_cur_message_layer
        #[batch_size, node_cnt] * [node_cnt, embed_dim] = [batch_size, embed_dim]
        #action_embed = tf.sparse_tensor_dense_matmul(tf.cast(self.action_select, tf.float32), cur_message_layer)
        action_embed = torch.matmul(self.action_select.type(torch.FloatTensor), cur_message_layer)

        # # [batch_size, embed_dim, embed_dim]
        #temp = tf.matmul(tf.expand_dims(action_embed, axis=2),tf.expand_dims(y_potential, axis=1))
        temp = torch.matmul(torch.unsqueeze(action_embed, dim=2),torch.unsqueeze(y_potential, dim=1))
        # [batch_size, embed_dim]
        #Shape = tf.shape(action_embed)
        Shape = action_embed.size()
        # [batch_size, embed_dim], first transform
        #embed_s_a = tf.reshape(tf.matmul(temp, tf.reshape(tf.tile(cross_product,[Shape[0],1]),[Shape[0],Shape[1],1])),Shape)
        embed_s_a = torch.reshape(torch.matmul(temp, torch.reshape(torch.tile(cross_product,[Shape[0],1]),\
                                                                   [Shape[0],Shape[1],1])),Shape)

        #[batch_size, 2 * embed_dim]
        last_output = embed_s_a

        if self.reg_hidden > 0:
            #[batch_size, 2*embed_dim] * [2*embed_dim, reg_hidden] = [batch_size, reg_hidden], dense
            #hidden = tf.matmul(embed_s_a, h1_weight)
            hidden = torch.matmul(embed_s_a, h1_weight)
            #[batch_size, reg_hidden]
            #last_output = tf.nn.relu(hidden)
            last_output = F.relu(hidden)

        # if reg_hidden == 0: ,[[batch_size, 2*embed_dim], [batch_size, aux_dim]] = [batch_size, 2*embed_dim+aux_dim]
        # if reg_hidden > 0: ,[[batch_size, reg_hidden], [batch_size, aux_dim]] = [batch_size, reg_hidden+aux_dim]
        #last_output = tf.concat([last_output, self.aux_input], 1)
        last_output = torch.concat([last_output, self.aux_input], 1)
        #if reg_hidden == 0: ,[batch_size, 2*embed_dim+aux_dim] * [2*embed_dim+aux_dim, 1] = [batch_size, 1]
        #if reg_hidden > 0: ,[batch_size, reg_hidden+aux_dim] * [reg_hidden+aux_dim, 1] = [batch_size, 1]
        #q_pred = tf.matmul(last_output, last_w)
        q_pred = torch.matmul(last_output, last_w)

        # TODO, losses and more
        
        return x
    
    def test_forward(self, node_input):

        x = torch.matmul(node_input, self.w_n2l)
        x = F.relu(x)
        
        if embeddingMethod == 1:    #'graphsage'

        return x


IndentationError: expected an indented block (2562620713.py, line 200)

## Misc

In [4]:
x_a, y_a = torch.ones((100, 2)), torch.ones(100)
x_b, y_b = torch.zeros((100, 2)), torch.zeros(100)


In [10]:
for i in range (4):
    
    M = Model(1, 1)


    numb_parameters = 0
    for param in M.parameters():
        numb_parameters += np.prod(list(param.shape))
    print(numb_parameters)

    print(M(torch.ones((5, 2))))



3
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)
3
tensor([[0.8910],
        [0.8910],
        [0.8910],
        [0.8910],
        [0.8910]], grad_fn=<ReluBackward0>)
3
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)
3
tensor([[0.1697],
        [0.1697],
        [0.1697],
        [0.1697],
        [0.1697]], grad_fn=<ReluBackward0>)


In [6]:
optimizer = optim.SGD(M.parameters(), lr=0.01, momentum=0.9)
criterion = nn.BCEWithLogitsLoss()


In [7]:
for i in range(100):
    optimizer.zero_grad()
    # Forward pass, backward pass, and optimize
    outputs = M(x_a)
    loss = criterion(outputs[:,0], y_a.float())
    loss.backward()
    optimizer.step()
    
    optimizer.zero_grad()
    # Forward pass, backward pass, and optimize
    outputs = M(x_b)
    loss = criterion(outputs[:,0], y_b.float())
    loss.backward()
    optimizer.step()

In [8]:
M(torch.ones(20,2))

tensor([[3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784]], grad_fn=<ReluBackward0>)

In [9]:
M(torch.zeros(20,2))

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)